In [2]:
import re

In [ ]:
sql_string = """
SELECT
    format('%20d', count(*)) AS source_string
  FROM
    `hca-hin-dev-cur-ops`.edwcr_base_views.cn_patient_address
  WHERE cn_patient_address.dw_last_update_date_time >= (
    SELECT
        max(etl_job_run.job_start_date_time) AS job_start_date_time
      FROM
        `hca-hin-dev-cur-ops`.edwcr_dmx_ac.etl_job_run
      WHERE upper(rtrim(etl_job_run.job_name)) = 'J_CN_PATIENT_ADDRESS'
  )
"""

In [ ]:
# print(sql_string)
regexp_subquery = re.compile(r"\(\s*SELECT\s*MAX\(etl_job_run\.job_start_date_time\).*FROM.*etl_job_run.*WHERE.*=.*'\s*\)",re.IGNORECASE | re.DOTALL)
regexp_repl_pattern = " tableload_start_time - INTERVAL 1 MINUTE "
converted_sql = re.sub(regexp_subquery, regexp_repl_pattern, sql_string)
print(converted_sql)

In [ ]:
sql_string = """
SELECT concat(format('%20d', count(*)), ', ') AS source_string
FROM
  (SELECT cn_patient_address_stg.*
   FROM `hca-hin-dev-cur-ops`.edwcr_staging.cn_patient_address_stg
   WHERE cn_patient_address_stg.nav_patient_id NOT IN
       (SELECT cn_patient_address.nav_patient_id
        FROM `hca-hin-dev-cur-ops`.edwcr.cn_patient_address)
     AND cn_patient_address_stg.dw_last_update_date_time <
       (SELECT max(etl_job_run.job_start_date_time) AS job_start_date_time
        FROM `hca-hin-dev-cur-ops`.edwcr_dmx_ac.etl_job_run
        WHERE upper(rtrim(etl_job_run.job_name)) = 'J_CN_PATIENT_ADDRESS' ) ) AS a
"""

In [ ]:
# print(sql_string)
core_dataset_list = ["edwcr"]
regexp_subquery = re.compile(r"((?:"+'|'.join(core_dataset_list)+")\.[a-z|0-9|_]+\s*[AS]*\s*(?!ON)[a-z|0-9|_]*)",re.IGNORECASE | re.DOTALL)
regexp_repl_pattern = r"\1 FOR SYSTEM_TIME AS OF TIMESTAMP(tableload_start_time,'US/Central')"
converted_sql = re.sub(regexp_subquery, regexp_repl_pattern, sql_string)
print(converted_sql)

In [ ]:
sql_string = """
SELECT concat(format('%20d', count(*)), ', ') AS source_string
FROM
  (SELECT cn_patient_address_stg.*
   FROM `hca-hin-dev-cur-ops`.edwcr_staging.cn_patient_address_stg
   WHERE cn_patient_address_stg.nav_patient_id NOT IN
       (SELECT cn_patient_address.nav_patient_id
        FROM `hca-hin-dev-cur-ops`.edwcr.cn_patient_address)
     AND cn_patient_address_stg.dw_last_update_date_time <
       (SELECT max(etl_job_run.job_start_date_time) AS job_start_date_time
        FROM `hca-hin-dev-cur-ops`.edwcr_dmx_ac.etl_job_run
        WHERE upper(rtrim(etl_job_run.job_name)) = 'J_CN_PATIENT_ADDRESS' ) ) AS a
"""

In [ ]:
# print(sql_string)
regexp_subquery = re.compile(r"([a-z|0-9|_]+.dw_last_update_date_time)\s*[<>=]\s*\(\s*SELECT\s*MAX\(etl_job_run\.job_start_date_time\).*FROM.*etl_job_run.*WHERE.*=.*'\s*\)",re.IGNORECASE | re.DOTALL)
regexp_repl_pattern = r"\1 = current_date('US/Central') "
# regexp_subquery.search(sql_string).group()
converted_sql = re.sub(regexp_subquery, regexp_repl_pattern, sql_string)
print(converted_sql)

In [ ]:
sql_string = """
SELECT 'J_CN_PATIENT_ADDRESS'||','|| cast(count(*) as varchar(20))||', ' as SOURCE_STRING 

FROM

(

SELECT *  FROM edwcr_staging.CN_PATIENT_ADDRESS_Stg

where Nav_Patient_Id not in ( Select Nav_Patient_Id from edwcr.CN_Patient_Address )

and DW_Last_Update_Date_Time < (SELECT MAX(Job_Start_Date_Time) as Job_Start_Date_Time FROM EDWCR_DMX_AC.ETL_JOB_RUN where Job_Name = 'J_CN_PATIENT_ADDRESS')

)A;
"""

In [ ]:
# print(sql_string)
staging_dataset_list = ["edwcr_staging"]
regexp_subquery = re.compile(r"(?:"+'|'.join(staging_dataset_list)+")\.([a-z|0-9|_]+)",re.IGNORECASE | re.DOTALL)
staging_table_find = regexp_subquery.search(sql_string)
if staging_table_find is None:
    staging_table = ""
else:
    staging_table = staging_table_find.groups()[0].lower()
print(staging_table)

In [ ]:
sql_string = " EOF"
regexp_subquery = re.compile(r"\bEOF\b",re.IGNORECASE | re.DOTALL)

regex_find = regexp_subquery.search(sql_string)
if not regex_find is None:
    print(regex_find)
else:
    print("EOF is not present")


In [ ]:
sql_string = """
edwcr.CN_PATIENT_ADDRESS_Stg
"""

In [ ]:
# print(sql_string)
core_dataset_list = ["edwcr"]
regexp_subquery = re.compile(r"(?:"+'|'.join(core_dataset_list)+")\.([a-z|0-9|_]+)",re.IGNORECASE | re.DOTALL)
core_table_find = regexp_subquery.search(sql_string)
if core_table_find is None:
    core_table = ""
else:
    core_table = core_table_find.groups()[0].lower()
print(core_table)

In [ ]:
sql_string = """
SELECT concat(format('%20d', count(*)), ',\t') AS source_string
FROM
  (SELECT row_number() OVER (
                             ORDER BY upper(trim(src.side_desc))) +
     (SELECT coalesce(max(ref_side.side_id), 0) AS id1
      FROM `hca-hin-dev-cur-ops`.edwcr.ref_side) AS side_id,
                            trim(src.side_desc) AS side_desc,
                            src.source_system_code,
                            datetime_trunc(current_datetime('US/Central'), SECOND) AS dw_last_update_date_time
   FROM
     (SELECT ref_side_stg.side_desc,
             ref_side_stg.source_system_code
      FROM `hca-hin-dev-cur-ops`.edwcr_staging.ref_side_stg
      WHERE upper(trim(ref_side_stg.side_desc)) NOT IN
          (SELECT upper(trim(ref_side.side_desc))
           FROM `hca-hin-dev-cur-ops`.edwcr.ref_side
           WHERE ref_side.side_desc IS NOT NULL ) ) AS src
   WHERE src.side_desc IS NOT NULL
     AND upper(rtrim(src.side_desc)) <> '' ) AS a
"""

In [ ]:
# print(sql_string)
core_dataset_list = ["edwcr"]
# regexp_subquery = re.compile(r"((?:"+'|'.join(core_dataset_list)+")\.[a-z|0-9|_]+\s*[AS]*\s*(?!ON)[a-z|0-9|_]*)",re.IGNORECASE | re.DOTALL)
regexp_subquery = re.compile(r"((?:"+'|'.join(core_dataset_list)+")\.[a-z|0-9|_]+\s*[AS]*\s*(?!ON)(?!WHERE)(?!INNER)(?!JOIN)[a-z|0-9|_]*)",re.IGNORECASE | re.DOTALL)
regexp_repl_pattern = r"\1 FOR SYSTEM_TIME AS OF TIMESTAMP(tableload_start_time,'US/Central')"
converted_sql = re.sub(regexp_subquery, regexp_repl_pattern, sql_string)
print(converted_sql)

In [ ]:
dict_a = {'oig': [{'v_host': 'nadcwpftp01.hca.corpad.net', 
'v_user': 'hca\\CorpSvcFtpIdmOig01', 
'v_port': 22, 
'v_pwd_secret_name': '***', 
'v_default_dir': 'Incoming_P2', 
'v_default_file_pattern': '^OIG-HCA\\_.*\\.csv$', 
'v_bqpytemplate': 'sFTP_to_BigQuery_PythonTemplate.py', 
'v_machine_type': 'n1-standard-4'},
{'v_host': 'nadcwqftp01.hca.corpad.net', 
'v_user': 'hca\\CorpSvcFtpIdmOig01', 
'v_port': 22, 
'v_pwd_secret_name': '***', 
'v_default_dir': 'Incoming_P3', 
'v_default_file_pattern': '^OIG-HCA\\_.*\\.csv$', 
'v_bqpytemplate': 'sFTP_to_BigQuery_PythonTemplate.py', 
'v_machine_type': 'n1-standard-4'}]
}
print(type(dict_a['oig']))
sftp_config = next(filter(lambda item: item['v_host'] == "nadcwpftp01.hca.corpad.net", dict_a['oig']),None)
# (item for items in dict_a['oig'] if item["v_host"] == "nadcwpftp01.hca.corpad.net")
print(sftp_config['v_default_dir'])

In [3]:
pattern_fmt_code = "HCA\_EPCS\_Active\_Inactive\_Providers\.20240826.*\.csv$"
gcs_object_path= "edwcompdata/srcfiles/drfirst/HCA_EPCS_Active_Inactive_Providers.20240826085001.csv"
re.search(pattern_fmt_code, gcs_object_path)

<re.Match object; span=(29, 82), match='HCA_EPCS_Active_Inactive_Providers.20240826085001>

In [ ]:
pattern_fmt_code = "^HCA\_EPCS\_Active\_Inactive\_Providers\.20240917\d+\.triggerfile\.csv$"
file_name = "HCA_EPCS_Active_Inactive_Providers.20240917085001.triggerfile.csv"
re.search(pattern_fmt_code, file_name)

In [5]:
pattern_fmt_code = "\d+"
gcs_object_path = "xrpswpdbspgu04.hca.corpad.net"
print(re.search(pattern_fmt_code, gcs_object_path, re.IGNORECASE))

<re.Match object; span=(12, 14), match='04'>


In [42]:
pattern_fmt_code = "cclog24mcrsnf(?:_[a-z]+|)\_D\_\d+\.csv$"
# gcs_object_path = "edwradata/srcfiles/govreports/american/csv/cclog24mcrsnf_py_d_38340.csv"
gcs_object_path = "edwradata/srcfiles/govreports/national/csv/cclog24mcrsnf_d_27630.csv"
print(re.search(pattern_fmt_code, gcs_object_path, re.IGNORECASE))

<re.Match object; span=(43, 68), match='cclog24mcrsnf_d_27630.csv'>


In [44]:
pattern_fmt_code = "cclog24mcrsnf(?:_[a-z]+|)\_d"
# gcs_object_path = "edwradata/srcfiles/govreports/american/csv/cclog24mcrsnf_py_d_38340.csv"
gcs_object_path = "cclog24mcrsnf_d_27630"
print(re.search(pattern_fmt_code, gcs_object_path, re.IGNORECASE))

<re.Match object; span=(0, 15), match='cclog24mcrsnf_d'>
